In [1]:
import numpy as np
import pandas as pd
# from pandas import Series, DataFrame 
import seaborn as sns
from matplotlib import pyplot as plt

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import warnings
from sklearn import metrics


warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.width', 10000)


In [2]:
df=pd.read_csv("d_before_encode.csv")

In [3]:
df.drop(["Unnamed: 0"], axis=1, inplace=True)
df.head()

,LEVEL,POSITION,EXPERIENCE,GENDER,COMPANY,WORK_TYPE,CITY,CURRENCY,SALARY_AVG_TL
0,Middle,BACK-END DEVELOPER,3 - 5 Yıl,Erkek,Startup,Hibrit,yurt_ici,₺ - Türk Lirası,15499
1,Junior,FRONT-END DEVELOPER,0 - 1 Yıl,Erkek,Startup,Hibrit,yurt_ici,₺ - Türk Lirası,8499
2,Middle,BACK-END DEVELOPER,1 - 3 Yıl,Erkek,Outsource,Remote,yurt_ici,₺ - Türk Lirası,15499
3,Junior,FULL STACK DEVELOPER,1 - 3 Yıl,Erkek,Startup,Hibrit,yurt_ici,₺ - Türk Lirası,10499
4,Junior,BACK-END DEVELOPER,0 - 1 Yıl,Erkek,Kurumsal,Geçici Hibrit,yurt_ici,₺ - Türk Lirası,5499


In [4]:
#EXPERIENCE degiskenini encod edilmeye hazır hale getirelim

In [5]:
df["EXPERIENCE"].unique()

array(['3 - 5 Yıl', '0 - 1 Yıl', '1 - 3 Yıl', '5 - 7 Yıl', '10 - 14 Yıl',
       '7 - 10 Yıl', '15 Yıl ve üzeri'], dtype=object)

In [6]:
for i in df[df["EXPERIENCE"]=="0 - 1 Yıl"].index:
    df["EXPERIENCE"][i] = 0
for i in df[df["EXPERIENCE"]=="1 - 3 Yıl"].index:
    df["EXPERIENCE"][i] = 1    
for i in df[df["EXPERIENCE"]=="3 - 5 Yıl"].index:
    df["EXPERIENCE"][i] = 2 
for i in df[df["EXPERIENCE"]=="5 - 7 Yıl"].index:
    df["EXPERIENCE"][i] = 3 
for i in df[df["EXPERIENCE"]=="7 - 10 Yıl"].index:
    df["EXPERIENCE"][i] = 4 
for i in df[df["EXPERIENCE"]=="10 - 14 Yıl"].index:
    df["EXPERIENCE"][i] = 5 
for i in df[df["EXPERIENCE"]=="15 Yıl ve üzeri"].index:
    df["EXPERIENCE"][i] = 6 

C:\Users\makif\AppData\Local\Temp\ipykernel_19152\2241603233.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["EXPERIENCE"][i] = 0
C:\Users\makif\AppData\Local\Temp\ipykernel_19152\2241603233.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["EXPERIENCE"][i] = 1
C:\Users\makif\AppData\Local\Temp\ipykernel_19152\2241603233.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["EXPERIENCE"][i] = 2
C:\Users\makif\AppData\Local

In [7]:
df["EXPERIENCE"].unique()

array([2, 0, 1, 3, 5, 4, 6], dtype=object)

In [8]:
def grab_col_names(dataframe, cat_th=15, car_th=20):
    """

    Veri setindeki kategorik, numerik ve kategorik fakat kardinal değişkenlerin isimlerini verir.
    Not: Kategorik değişkenlerin içerisine numerik görünümlü kategorik değişkenler de dahildir.

    Parameters
    ------
        dataframe: dataframe
                Değişken isimleri alınmak istenilen dataframe
        cat_th: int, optional
                numerik fakat kategorik olan değişkenler için sınıf eşik değeri
        car_th: int, optinal
                kategorik fakat kardinal değişkenler için sınıf eşik değeri

    Returns
    ------
        cat_cols: list
                Kategorik değişken listesi
        num_cols: list
                Numerik değişken listesi
        cat_but_car: list
                Kategorik görünümlü kardinal değişken listesi

    Examples
    ------
        import seaborn as sns
        df = sns.load_dataset("iris")
        print(grab_col_names(df))


    Notes
    ------
        cat_cols + num_cols + cat_but_car = toplam değişken sayısı
        num_but_cat cat_cols'un içerisinde.
        Return olan 3 liste toplamı toplam değişken sayısına eşittir: cat_cols + num_cols + cat_but_car = değişken sayısı

    """

    # cat_cols, cat_but_car
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtypes == "O"]
    num_but_cat = [col for col in dataframe.columns if dataframe[col].nunique() < cat_th and
                   dataframe[col].dtypes != "O"]
    cat_but_car = [col for col in dataframe.columns if dataframe[col].nunique() > car_th and
                   dataframe[col].dtypes == "O"]
    cat_cols = cat_cols + num_but_cat
    cat_cols = [col for col in cat_cols if col not in cat_but_car]

    # num_cols
    num_cols = [col for col in dataframe.columns if dataframe[col].dtypes != "O"]
    num_cols = [col for col in num_cols if col not in num_but_cat]

    print(f"Observations: {dataframe.shape[0]}")
    print(f"Variables: {dataframe.shape[1]}")
    print(f'cat_cols: {len(cat_cols)}')
    print(f'num_cols: {len(num_cols)}')
    print(f'cat_but_car: {len(cat_but_car)}')
    print(f'num_but_cat: {len(num_but_cat)}')
    return cat_cols, num_cols, cat_but_car, num_but_cat

In [9]:
cat_cols, num_cols, cat_but_car, num_but_cat= grab_col_names(df)

Observations: 5026
Variables: 9
cat_cols: 8
num_cols: 1
cat_but_car: 0
num_but_cat: 0


In [10]:
cat_cols

['LEVEL',
 'POSITION',
 'EXPERIENCE',
 'GENDER',
 'COMPANY',
 'WORK_TYPE',
 'CITY',
 'CURRENCY']

In [11]:
df[cat_cols].nunique()

LEVEL          3
POSITION      15
EXPERIENCE     7
GENDER         2
COMPANY        9
WORK_TYPE      5
CITY           2
CURRENCY       4
dtype: int64

In [12]:
# Sınıflar arası fark varsa (ordinal kategorik değişkenler)- Label Encoding
#le_cols = ["LEVEL","EXPERIENCE"]

# Sınıflar arası fark yoksa (nominal kategorik değişkenler) - One Hot Encoding
#ohe_cols = ["POSITION", "GENDER", "COMPANY", "WORK_TYPE", "CITY"]

In [13]:
#label encoding yapalım [LEVEL] ve [EXPERIENCE] icin
labelencoder = LabelEncoder()

In [14]:
df["LEVEL"] = labelencoder.fit_transform(df["LEVEL"])

In [15]:
df["EXPERIENCE"] = labelencoder.fit_transform(df["EXPERIENCE"])

In [16]:
cat_cols=["POSITION", "GENDER", "COMPANY", "WORK_TYPE", "CITY", "CURRENCY"]
cat_cols

['POSITION', 'GENDER', 'COMPANY', 'WORK_TYPE', 'CITY', 'CURRENCY']

In [17]:
#kategorik degiskenlerde bicimlendirme yapıyoruz
def one_hot_encoder(dataframe, categorical_cols, drop_first=False):
    dataframe = pd.get_dummies(dataframe, columns=categorical_cols, drop_first=drop_first)
    return dataframe
#drop_first=True yaparsak label encoder olarak kullanabiliriz,iki sınıflı kategorik degiskenleri de label encoder a sokabiliyor olacak

In [18]:
df = one_hot_encoder(df, cat_cols, drop_first=True)

In [19]:
df.head()

,LEVEL,EXPERIENCE,SALARY_AVG_TL,POSITION_CTO,POSITION_DATA SCIENTIST,POSITION_DATABASE ADMIN,POSITION_DEVOPS ENGINEER,POSITION_EMBEDDED SOFTWARE DEVELOPER,POSITION_FRONT-END DEVELOPER,POSITION_FULL STACK DEVELOPER,...,COMPANY_Startup,COMPANY_Yazılım & Teknoloji,WORK_TYPE_Geçici Remote,WORK_TYPE_Hibrit,WORK_TYPE_Ofis,WORK_TYPE_Remote,CITY_yurt_ici,CURRENCY_£ - Sterlin,CURRENCY_€ - Euro,CURRENCY_₺ - Türk Lirası
0,1,2,15499,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,0,0,1
1,0,0,8499,0,0,0,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1
2,1,1,15499,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,1
3,0,1,10499,0,0,0,0,0,0,1,...,1,0,0,1,0,0,1,0,0,1
4,0,0,5499,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [20]:
df.columns

Index(['LEVEL', 'EXPERIENCE', 'SALARY_AVG_TL', 'POSITION_CTO', 'POSITION_DATA SCIENTIST', 'POSITION_DATABASE ADMIN', 'POSITION_DEVOPS ENGINEER', 'POSITION_EMBEDDED SOFTWARE DEVELOPER', 'POSITION_FRONT-END DEVELOPER', 'POSITION_FULL STACK DEVELOPER', 'POSITION_GAME DEVELOPER', 'POSITION_MOBILE APPLICATION DEVELOPER', 'POSITION_OTHERS', 'POSITION_QA / TEST AUTOMATION DEVELOPER', 'POSITION_SOFTWARE ARCHITECT', 'POSITION_SOFTWARE DEVELOPMENT MANAGER', 'POSITION_TEAM / TECH LEAD', 'GENDER_Kadın', 'COMPANY_Dijital / Reklam Ajansı', 'COMPANY_E-Ticaret', 'COMPANY_Fintech / Finans', 'COMPANY_Kurumsal', 'COMPANY_Outsource', 'COMPANY_Oyun', 'COMPANY_Startup', 'COMPANY_Yazılım & Teknoloji', 'WORK_TYPE_Geçici Remote', 'WORK_TYPE_Hibrit', 'WORK_TYPE_Ofis', 'WORK_TYPE_Remote', 'CITY_yurt_ici', 'CURRENCY_£ - Sterlin', 'CURRENCY_€ - Euro', 'CURRENCY_₺ - Türk Lirası'], dtype='object')

In [21]:
def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### Tail #####################")
    print(dataframe.tail(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df)

##################### Shape #####################
(5026, 34)
##################### Types #####################
LEVEL                                      int32
EXPERIENCE                                 int32
SALARY_AVG_TL                              int64
POSITION_CTO                               uint8
POSITION_DATA SCIENTIST                    uint8
POSITION_DATABASE ADMIN                    uint8
POSITION_DEVOPS ENGINEER                   uint8
POSITION_EMBEDDED SOFTWARE DEVELOPER       uint8
POSITION_FRONT-END DEVELOPER               uint8
POSITION_FULL STACK DEVELOPER              uint8
POSITION_GAME DEVELOPER                    uint8
POSITION_MOBILE APPLICATION DEVELOPER      uint8
POSITION_OTHERS                            uint8
POSITION_QA / TEST AUTOMATION DEVELOPER    uint8
POSITION_SOFTWARE ARCHITECT                uint8
POSITION_SOFTWARE DEVELOPMENT MANAGER      uint8
POSITION_TEAM / TECH LEAD                  uint8
GENDER_Kadın                               uint8
COMPANY

In [25]:
#degiskenlerin sırasını ayarlama
# Değişkenler için bir liste oluşturalım
cols = df.columns.tolist()
cols
# Move the third (2nd index) element to the last position
cols = cols[:2] + cols[3:] + cols[2:3]
cols
# Apply it to the df
df = df[cols]
df.head()

,LEVEL,EXPERIENCE,POSITION_DATA SCIENTIST,POSITION_DATABASE ADMIN,POSITION_DEVOPS ENGINEER,POSITION_EMBEDDED SOFTWARE DEVELOPER,POSITION_FRONT-END DEVELOPER,POSITION_FULL STACK DEVELOPER,POSITION_GAME DEVELOPER,POSITION_MOBILE APPLICATION DEVELOPER,...,WORK_TYPE_Geçici Remote,WORK_TYPE_Hibrit,WORK_TYPE_Ofis,WORK_TYPE_Remote,CITY_yurt_ici,CURRENCY_£ - Sterlin,CURRENCY_€ - Euro,CURRENCY_₺ - Türk Lirası,SALARY_AVG_TL,POSITION_CTO
0,1,2,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,1,15499,0
1,0,0,0,0,0,0,1,0,0,0,...,0,1,0,0,1,0,0,1,8499,0
2,1,1,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,1,15499,0
3,0,1,0,0,0,0,0,1,0,0,...,0,1,0,0,1,0,0,1,10499,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,5499,0


In [ ]:
#buraya kadar olan yer encode edilmiş df

In [26]:
df.to_csv("d_encoded_sirali.csv")